Data Exploration - NFL Data
Evan Hughes

In [5]:
import pandas as pd

# import NFL datasets
nfl_games = pd.read_csv("https://raw.githubusercontent.com/nfl-football-ops/Big-Data-Bowl/master/Data/games.csv")
nfl_plays = pd.read_csv("https://raw.githubusercontent.com/nfl-football-ops/Big-Data-Bowl/master/Data/plays.csv")

In [6]:
# Data Exploration

## dimensions of dataset
nfl_games.shape
nfl_games.columns
nfl_games.head(5)

,season,week,gameDate,gameId,gameTimeEastern,HomeScore,VisitorScore,homeTeamAbbr,visitorTeamAbbr,homeDisplayName,...,Stadium,Location,StadiumType,Turf,GameLength,GameWeather,Temperature,Humidity,WindSpeed,WindDirection
0,2017,1,09/10/2017,2017091004,13:00:00,35,23,DET,ARI,Detroit Lions,...,Ford Field,"Detroit, MI",Indoors,Field Turf,03:26:00,Controlled Climate,68.0,70.0,NaN,NaN
1,2017,1,09/07/2017,2017090700,20:30:00,27,42,NE,KC,New England Patriots,...,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,03:37:00,Clear and warm,63.0,77.0,8,SW
2,2017,1,09/10/2017,2017091003,13:00:00,18,21,CLE,PIT,Cleveland Browns,...,FirstEnergy,"Cleveland, Ohio",Outdoor,Grass,02:59:00,Sunny,63.0,53.0,11,East
3,2017,1,09/10/2017,2017091002,13:00:00,0,20,CIN,BAL,Cincinnati Bengals,...,Paul Brown Stadium,"Cincinnati, Ohio",Outdoor,UBU Sports Speed S5-M,02:57:00,Sunny,68.0,43.0,9,SE
4,2017,1,09/10/2017,2017091000,13:00:00,21,12,BUF,NYJ,Buffalo Bills,...,New Era Field,Orchard Park NY,Outdoors,A-Turf Titan,02:54:00,Sun & clouds,65.0,53.0,6,NNE


In [7]:
nyg_games = nfl_games.loc[(nfl_games['homeTeamAbbr'] == "NYG") | (nfl_games['visitorTeamAbbr'] == "NYG")].copy()

nyg_games_tracking = pd.DataFrame()
for game in nyg_games['gameId']:
    game_tracking = pd.read_csv("https://raw.githubusercontent.com/nfl-football-ops/Big-Data-Bowl/master/Data/tracking_gameId_{}.csv".format(game))
    game_tracking['gameplayId'] = game_tracking['gameId'].astype(str) + "-" + game_tracking['playId'].astype(str)
    nyg_games_tracking = nyg_games_tracking.append(game_tracking, ignore_index=True)

In [10]:
nfl_plays['gameplayId'] = nfl_plays['gameId'].astype(str) + "-" + nfl_plays['playId'].astype(str)
nyg_plays = nfl_plays[nfl_plays['gameplayId'].isin(nyg_games_tracking['gameplayId'].unique())].copy()
nyg_plays.loc[:, 'gameplayId'] = nyg_plays['gameId'].astype(str) + "-" + nfl_plays['playId'].astype(str)

In [13]:
nyg_plays.loc[:, 'gameplaytimeId'] = nyg_plays['quarter'] * (pd.to_datetime('1900-01-01 00:15:00') - pd.to_datetime(nyg_plays['GameClock'], format='%M:%S:%f'))
nyg_plays.head(5)

,gameId,playId,quarter,GameClock,down,yardsToGo,possessionTeam,yardlineSide,yardlineNumber,offenseFormation,...,isSTPlay,SpecialTeamsPlayType,KickReturnYardage,PassLength,PassResult,YardsAfterCatch,PlayResult,playDescription,gameplayId,gameplaytimeId
1821,2017091012,44,1,15:00:00,0,0,NYG,NYG,35.0,NaN,...,True,Kickoff,25.0,NaN,NaN,NaN,41,A.Rosas kicks 66 yards from NYG 35 to DAL -1. ...,2017091012-44,00:00:00
1822,2017091012,66,1,14:55:00,1,10,DAL,DAL,24.0,SINGLEBACK,...,False,NaN,NaN,NaN,R,NaN,6,(14:55) D.Prescott scrambles left end pushed o...,2017091012-66,00:00:05
1823,2017091012,87,1,14:25:00,2,4,DAL,DAL,30.0,SINGLEBACK,...,False,NaN,NaN,NaN,NaN,NaN,3,(14:25) E.Elliott right guard to DAL 33 for 3 ...,2017091012-87,00:00:35
1824,2017091012,108,1,13:45:00,3,1,DAL,DAL,33.0,SHOTGUN,...,False,NaN,NaN,NaN,NaN,NaN,1,(13:45) (Shotgun) D.Prescott up the middle to ...,2017091012-108,00:01:15
1825,2017091012,129,1,13:06:00,1,10,DAL,DAL,34.0,SINGLEBACK,...,False,NaN,NaN,NaN,NaN,NaN,3,(13:06) E.Elliott right tackle to DAL 37 for 3...,2017091012-129,00:01:54


In [ ]:
row = 0
drive_number = 0
for index in nyg_plays.index:
    if row >= (nyg_plays.shape[0] - 1):
        nyg_plays.loc[(nyg_plays.index[row]), 'driveId'] = drive_number    
    elif nyg_plays.loc[(nyg_plays.index[row]), 'possessionTeam'] == nyg_plays.loc[(nyg_plays.index[row+1]), 'possessionTeam']:
        nyg_plays.loc[(nyg_plays.index[row]), 'driveId'] = drive_number 
    else:
        nyg_plays.loc[(nyg_plays.index[row]), 'driveId'] = drive_number
        drive_number = drive_number + 1
    row = row + 1
    
nyg_plays['gameplaydriveId'] = nyg_plays['gameplayId'].astype(str) + "-" + nyg_plays['driveId'].astype(str)

In [14]:
nyg_plays_off = nyg_plays.loc[nyg_plays['possessionTeam'] == "NYG"]
nyg_plays_off_tracking = nyg_games_tracking.loc[nyg_games_tracking['gameplayId'].isin(nyg_plays_off['gameplayId'].unique())]
nyg_plays_off_tracking.head(5)

,time,x,y,s,dis,dir,event,nflId,displayName,jerseyNumber,team,frame.id,gameId,playId,gameplayId
0,2017-09-11T00:31:28Z,79.47,43.12,0.37,0.00,283.03,NaN,2495159.0,Dwayne Harris,17.0,away,1,2017091012,44,2017091012-44
1,2017-09-11T00:31:29Z,79.47,43.12,0.61,0.00,279.91,NaN,2495159.0,Dwayne Harris,17.0,away,2,2017091012,44,2017091012-44
2,2017-09-11T00:31:29Z,79.47,43.12,0.84,0.00,275.20,NaN,2495159.0,Dwayne Harris,17.0,away,3,2017091012,44,2017091012-44
3,2017-09-11T00:31:29Z,79.46,43.12,1.07,0.00,272.42,NaN,2495159.0,Dwayne Harris,17.0,away,4,2017091012,44,2017091012-44
4,2017-09-11T00:31:29Z,79.46,43.12,1.30,0.01,270.77,NaN,2495159.0,Dwayne Harris,17.0,away,5,2017091012,44,2017091012-44
